In [1]:
import requests
import time
import pymysql
import os
from random import uniform
from bs4 import BeautifulSoup
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager

In [2]:
def rest_get(apilink):
    print(apilink)
    resp = requests.get(apilink)
    if resp.status_code != 200:
        # This means something went wrong.
        raise Exception('GET {} {}'.format(apilink, resp.status_code))
    else:
        return resp

### API approach, has limits per day - not worth it, continue to full scraping approach

In [3]:
'''
links = ['https://github.com/poloclub/cnn-explainer']
github = 'https://api.github.com/'
gituser = 'https://github-user.now.sh/?username='
    
for l in links:
    repo_split = l.split('/')
    owner, repo = repo_split[-2], repo_split[-1]
    print('Requestiong: ',owner, repo)
    resp = rest_get(f'{github}repos/{owner}/{repo}/contributors')
    for user in resp.json():
        print(f'user "{user["login"]}" has {user["contributions"]} contributions')
        user_info = rest_get(gituser+user["login"])
        print(f"Name: {user_info['name']}, Description: {user_info['description']}")
'''
    

'\nlinks = [\'https://github.com/poloclub/cnn-explainer\']\ngithub = \'https://api.github.com/\'\ngituser = \'https://github-user.now.sh/?username=\'\n    \nfor l in links:\n    repo_split = l.split(\'/\')\n    owner, repo = repo_split[-2], repo_split[-1]\n    print(\'Requestiong: \',owner, repo)\n    resp = rest_get(f\'{github}repos/{owner}/{repo}/contributors\')\n    for user in resp.json():\n        print(f\'user "{user["login"]}" has {user["contributions"]} contributions\')\n        user_info = rest_get(gituser+user["login"])\n        print(f"Name: {user_info[\'name\']}, Description: {user_info[\'description\']}")\n'

### Saving author contributions to txt file

In [4]:
conn=pymysql.connect(host='mysql24.ezhostingserver.com', port=3306, user='krw',password='Y>!V@N_26@]cfJ7(')
cursor=conn.cursor()

In [5]:

repos = []
cursor.execute("SELECT DISTINCT krw.repo.repo_url FROM krw.paper \
INNER JOIN krw.repo ON krw.repo.paper_url = krw.paper.paper_url \
WHERE krw.paper.makg_sameAs <> 'notfound'")
for r in cursor:
    repos.append(list(r)[0])
'''
cursor.execute("SELECT krw.repo.paper_url, repo_url, author \
               FROM krw.repo INNER JOIN krw.paper_author \
               ON krw.repo.paper_url=krw.paper_author.paper_url limit 5000;")#("SELECT * FROM krw.training_view limit 10;")
for r in cursor:
    data.append(list(r))
'''

'\ncursor.execute("SELECT krw.repo.paper_url, repo_url, author                FROM krw.repo INNER JOIN krw.paper_author                ON krw.repo.paper_url=krw.paper_author.paper_url limit 5000;")#("SELECT * FROM krw.training_view limit 10;")\nfor r in cursor:\n    data.append(list(r))\n'

In [6]:
len(repos)

11266

In [7]:
filename = 'scraped_data.txt'
if not os.path.exists(filename):
    file = open(filename, 'w+')
    file.close()

In [8]:
# Remove repos we already saved into file
with open(filename) as fp:
    line = fp.readline().split()
    while line:
        current_repo = line[0]
        if current_repo in repos:
            repos.remove(current_repo)
        line = fp.readline().split()
len(repos)

9760

In [9]:
repos.index('https://github.com/facebook/fb.resnet.torch')

ValueError: 'https://github.com/facebook/fb.resnet.torch' is not in list

In [11]:
missed = []

In [12]:
driver = webdriver.Chrome(ChromeDriverManager().install()) # Firefox()

for repo in repos[1401:1700]:
    print(repo) # github repo and authors
    driver.get(f'{repo}/graphs/contributors')
    time.sleep(12+uniform(1, 3)) # wait for contribution graph to load
    website = BeautifulSoup(driver.page_source, 'html')
    boxes = website.find_all('h3', {'class':'border-bottom p-2 lh-condensed'}) 
    if not len(boxes):
        print('NOT FOUND/LOADED')
        missed.append(repo)
    else:
        for user in boxes[0:15]:   # take just top 15 contributors, some repos might have 100+ contributors
            username = user.find('a', {'class':'text-normal', 'data-hovercard-type':'user'}).text.strip()
            info_line = user.find_all(('span', {'class':'cmeta'}))
            scrape = rest_get(f'https://github.com/{username}')
            soup = BeautifulSoup(scrape.content, 'html.parser')
            time.sleep(uniform(1, 2)) # adding more delays to avoid being blocked by github
            name = soup.find('span', class_='p-name vcard-fullname d-block overflow-hidden').text.strip()
            bio = soup.find('div', class_='p-note user-profile-bio mb-2 js-user-profile-bio').text.strip()
            print(f'name: {name}, bio: {bio}')

            commits = info_line[1].find('a').text.strip().split()[0]
            added = info_line[1].find('span', {'class':"text-green text-normal"}).text.strip().split()[0]
            deleted = info_line[1].find('span', {'class':"text-red text-normal"}).text.strip().split()[0]
            if name=="":
                name = "NULL"
            else:
                name = name.replace(" ", "_")
            with open(filename, 'a') as file:
                file.write(f'{repo} {username} {name} {commits} {added} {deleted}\n')
            print(f'User {username}: commits {commits} , changed lines + {added} - {deleted}')
    print()

[WDM] - Looking for [chromedriver 81.0.4044.138 linux64] driver in cache 
[WDM] - File found in cache by path [/home/kanter/.wdm/drivers/chromedriver/81.0.4044.138/linux64/chromedriver]


https://github.com/juenkhaw/action_recognition_project1
https://github.com/juenkhaw
name: Juen Khaw, bio: 
User juenkhaw: commits 127 , changed lines + 70,142 - 11,212

https://github.com/Kadenze/siamese_net
NOT FOUND/LOADED

https://github.com/eecn/Hyperspectral-Classification
https://github.com/eecn
name: wk, bio: 
User eecn: commits 17 , changed lines + 2,672 - 98

https://github.com/hhexiy/interpretese
NOT FOUND/LOADED

https://github.com/ahhan02/darknet-alex
NOT FOUND/LOADED

https://github.com/madras-simulator/Multi-Agent-Particle-Environment
NOT FOUND/LOADED

https://github.com/aiff22/Decision-Stream
NOT FOUND/LOADED

https://github.com/tinagui/Swarm-Intelligence-in-Bioinformatics
https://github.com/tinagui
name: ✿ iCki3 TiNa ✿, bio: Computer Scientist
User tinagui: commits 102 , changed lines + 2,089 - 1,412

https://github.com/hltfbk/CROMER
NOT FOUND/LOADED

https://github.com/kirillermolov/ChatBot
NOT FOUND/LOADED

https://github.com/tdmeeste/SparseSeqModels
NOT FOUND/LOADED


NOT FOUND/LOADED

https://github.com/fitushar/Cyclical-Learning-Rates-for-Training-Neural-Networks-With-Unbalanced-Data-Sets
NOT FOUND/LOADED

https://github.com/vik-goel/MOREL
NOT FOUND/LOADED

https://github.com/warmspringwinds/pytorch-segmentation-detection
https://github.com/warmspringwinds
name: Daniil Pakhomov, bio: Phd student at JHU. Research interests: Image Classification, Image Segmentation, Face Detection and Face Recognition mostly based on Deep Learning.
User warmspringwinds: commits 253 , changed lines + 56,625 - 19,329
https://github.com/peteflorence
name: Pete Florence, bio: Robotics and computer vision,
PhD student at MIT
https://twitter.com/peteflorence
User peteflorence: commits 2 , changed lines + 9 - 10
https://github.com/Randl
name: Evgeniy Zheltonozhskiy, bio: 
User Randl: commits 2 , changed lines + 3 - 2
https://github.com/erasaur
name: Mark Lee, bio: 
User erasaur: commits 1 , changed lines + 25 - 25

https://github.com/paulorauber/nn
NOT FOUND/LOADED

https:

name: Lisa Zhang, bio: 
User xuexue: commits 5 , changed lines + 28 - 6
https://github.com/gregr
name: Greg Rosenblatt, bio: people machines, not machine people
User gregr: commits 1 , changed lines + 3 - 3

https://github.com/syiin/human_protein_atlas
NOT FOUND/LOADED

https://github.com/XinlueLiu/TensorFlow-classifiers
NOT FOUND/LOADED

https://github.com/sminer3/duplicate_statement_detection
NOT FOUND/LOADED

https://github.com/xunhen/py-faster-rcnn-wjc
NOT FOUND/LOADED

https://github.com/renqianluo/NAO
https://github.com/renqianluo
name: Renqian Luo, bio: Joint Ph.D. @ MSRA & USTC
User renqianluo: commits 31 , changed lines + 158,094 - 615
https://github.com/prabhant
name: prabhant, bio: Research Engineer
User prabhant: commits 1 , changed lines + 1 - 1

https://github.com/Ereebay/CV-Documents
NOT FOUND/LOADED

https://github.com/TianchengQ/FCN
NOT FOUND/LOADED

https://github.com/vermaMachineLearning/FGSD
NOT FOUND/LOADED

https://github.com/markvdw/convgp
NOT FOUND/LOADED

https

NOT FOUND/LOADED

https://github.com/ColdenChan/GAT_D
NOT FOUND/LOADED

https://github.com/awjuliani/Pix2Pix-Film
https://github.com/awjuliani
name: Arthur Juliani, bio: 
User awjuliani: commits 13 , changed lines + 547 - 27
https://github.com/dnlcrl
name: , bio: Burning up with curiosity. MSc in Computer Engineering; pythonista, dad and doodler
User dnlcrl: commits 1 , changed lines + 2 - 2

https://github.com/jojonki/key-value-memory-networks
https://github.com/jojonki
name: Junki Ohmura, bio: Sony Researcher. Previously, visiting researcher at Language Technologies Institute, Carnegie Mellon University.
Email: junki dot ohmura at gmail.
User jojonki: commits 139 , changed lines + 202,424 - 198,993
https://github.com/samhavens
name: Sam Havens, bio: Just all over the place. 

@sam_havens on twitter.
User samhavens: commits 2 , changed lines + 11 - 3

https://github.com/raycad/object-detection
NOT FOUND/LOADED

https://github.com/DanishShah/DeepDiagnosis
NOT FOUND/LOADED

https://gith

NOT FOUND/LOADED

https://github.com/ymollard/APEX
https://github.com/ymollard
name: Yoan Mollard, bio: Engineer roboticist @Bordeaux-INP, founder of @arbalet-project, entrepreneur, and opensource enthusiast! Ex @flowersteam
User ymollard: commits 300 , changed lines + 23,642 - 19,095
https://github.com/sebastien-forestier
name: Sébastien Forestier, bio: 
User sebastien-forestier: commits 108 , changed lines + 9,006 - 3,728
https://github.com/dashdashzako
name: Damien Caselli, bio: Web Frontend Engineer
User dashdashzako: commits 13 , changed lines + 297 - 28

https://github.com/jesse1029/Fake-Face-Images-Detection-Tensorflow
https://github.com/jesse1029
name: Chih-Chung Hsu, bio: 
User jesse1029: commits 14 , changed lines + 9,068 - 3,899

https://github.com/fwang91/residual-attention-network
https://github.com/wangfei666
name: , bio: 
User wangfei666: commits 7 , changed lines + 22,336 - 20
https://github.com/fwang91
name: Fei Wang, bio: wangfei.info
User fwang91: commits 4 , changed

name: Youngjung, bio: 
User youngjung: commits 1 , changed lines + 1,321 - 0
https://github.com/us
name: , bio: GSoC'19 @tensorflow
User us: commits 1 , changed lines + 13 - 3
https://github.com/ClovaAIAdmin
name: Admin of Clova AI Open Source, bio: 
User ClovaAIAdmin: commits 1 , changed lines + 106 - 0

https://github.com/karthickrajas/contextualMAB
NOT FOUND/LOADED

https://github.com/network-embeddings/temporal_embedding_matching
NOT FOUND/LOADED

https://github.com/alankarj/robust_nlp
https://github.com/bhargaviparanjape
name: Bhargavi Paranjape, bio: PhD student @uwnlp
User bhargaviparanjape: commits 6 , changed lines + 12,111 - 0
https://github.com/alankarj
name: , bio: 
User alankarj: commits 6 , changed lines + 118 - 1

https://github.com/2xic/notebooks
NOT FOUND/LOADED

https://github.com/AliLotfi92/Disentangling_by_Factorising
https://github.com/AliLotfi92
name: Ali Lotfi Rezaabad, bio: PhD Student,
User AliLotfi92: commits 13 , changed lines + 229 - 35

https://github.com/l

In [ ]:
driver = webdriver.Firefox() # 

for repo in repos[1701:2400]:
    print(repo) # github repo and authors
    driver.get(f'{repo}/graphs/contributors')
    time.sleep(12+uniform(1, 3)) # wait for contribution graph to load
    website = BeautifulSoup(driver.page_source, 'html')
    boxes = website.find_all('h3', {'class':'border-bottom p-2 lh-condensed'}) 
    if not len(boxes):
        print('NOT FOUND/LOADED')
        missed.append(repo)
    else:
        for user in boxes[0:15]:   # take just top 15 contributors, some repos might have 100+ contributors
            username = user.find('a', {'class':'text-normal', 'data-hovercard-type':'user'}).text.strip()
            info_line = user.find_all(('span', {'class':'cmeta'}))
            scrape = rest_get(f'https://github.com/{username}')
            soup = BeautifulSoup(scrape.content, 'html.parser')
            time.sleep(uniform(1, 2)) # adding more delays to avoid being blocked by github
            name = soup.find('span', class_='p-name vcard-fullname d-block overflow-hidden').text.strip()
            bio = soup.find('div', class_='p-note user-profile-bio mb-2 js-user-profile-bio').text.strip()
            print(f'name: {name}, bio: {bio}')

            commits = info_line[1].find('a').text.strip().split()[0]
            added = info_line[1].find('span', {'class':"text-green text-normal"}).text.strip().split()[0]
            deleted = info_line[1].find('span', {'class':"text-red text-normal"}).text.strip().split()[0]
            if name=="":
                name = "NULL"
            else:
                name = name.replace(" ", "_")
            with open(filename, 'a') as file:
                file.write(f'{repo} {username} {name} {commits} {added} {deleted}\n')
            print(f'User {username}: commits {commits} , changed lines + {added} - {deleted}')
    print()

https://github.com/bamsumit/slayerPytorch
https://github.com/bamsumit
name: , bio: 
User bamsumit: commits 104 , changed lines + 5,187,112 - 503,777
https://github.com/luca-della-vedova
name: Luca Della Vedova, bio: 
User luca-della-vedova: commits 49 , changed lines + 25,508 - 8,887
https://github.com/sunpengfei1122
name: , bio: 
User sunpengfei1122: commits 1 , changed lines + 2 - 2
https://github.com/magehrig
name: Mathias Gehrig, bio: 
User magehrig: commits 1 , changed lines + 1 - 1
https://github.com/jethrokuan
name: Jethro Kuan, bio: 
User jethrokuan: commits 1 , changed lines + 2 - 2
https://github.com/fabien-colonnier
name: , bio: Researcher


avatar created with https://personas.draftbit.com/
User fabien-colonnier: commits 1 , changed lines + 673 - 513

https://github.com/BY571/DQN-Atari-Agents
NOT FOUND/LOADED

https://github.com/liuzhuang13/DenseNet
https://github.com/liuzhuang13
name: Zhuang Liu, bio: 
User liuzhuang13: commits 50 , changed lines + 684 - 223
https://github

https://github.com/jiny2001
name: Jin Yamanaka, bio: real-time devices, computer vision, distributed system and robotics engineer.
twitter: Jiny2001
User jiny2001: commits 111 , changed lines + 7,353 - 5,867
https://github.com/tehtea
name: Jing Wei, bio: Building stuff
User tehtea: commits 6 , changed lines + 431 - 41
https://github.com/KnutZuidema
name: Knut Zuidema, bio: Computer Science student at FH Aachen | Backend Developer at @grid-x
User KnutZuidema: commits 2 , changed lines + 1,368 - 1,349
https://github.com/XiaotianM
name: , bio: 
User XiaotianM: commits 1 , changed lines + 25 - 9

https://github.com/DivJAth/DeepLearning5922
NOT FOUND/LOADED

https://github.com/bxck75/piss-ant-pix2pix
NOT FOUND/LOADED

https://github.com/SDuari/sCAKE-and-LAKE
NOT FOUND/LOADED

https://github.com/EggplantElf/sclem2017-tagger
NOT FOUND/LOADED

https://github.com/Hwang64/PSIS
NOT FOUND/LOADED

https://github.com/gcwl/pytorch-pix2pix
NOT FOUND/LOADED

https://github.com/tdozat/Parser
https://git

NOT FOUND/LOADED

https://github.com/hjgithub1/reinforcement-learning-notes
NOT FOUND/LOADED

https://github.com/xiaolonw/adversarial-frcnn
NOT FOUND/LOADED

https://github.com/ericlee0803/GP_Derivatives
NOT FOUND/LOADED

https://github.com/nekonaute/roborobo3
NOT FOUND/LOADED

https://github.com/joemzhao/tweets-retriever
NOT FOUND/LOADED

https://github.com/wohnjayne/eda-suite
NOT FOUND/LOADED

https://github.com/hussam123/Text-Summarization
NOT FOUND/LOADED

https://github.com/kunalrdeshmukh/End-to-end-compression
https://github.com/kunalrdeshmukh
name: Kunal Deshmukh, bio: Computer Science graduate student at San Jose State University.  Intern at Ericsson.
User kunalrdeshmukh: commits 39 , changed lines + 5,907 - 1,967

https://github.com/DrChainsaw/neuralODE4j
https://github.com/DrChainsaw
name: , bio: 
User DrChainsaw: commits 199 , changed lines + 30,080 - 10,851

https://github.com/Tsejing/object_detection
https://github.com/Tsejing
name: , bio: 人工智障、机器瞎学、模式混淆、数据掩埋、计算机幻觉  
Simul

In [14]:
len(missed)

80

### Getting list of repositories and authors from the file

In [10]:
data = []
cursor.execute("SELECT krw.repo.paper_url, repo_url, author \
               FROM krw.repo INNER JOIN krw.paper_author \
               ON krw.repo.paper_url=krw.paper_author.paper_url limit 10000;")#("SELECT * FROM krw.training_view limit 10;")
for r in cursor:
    data.append(list(r))

In [11]:
author_dict = {}
# Version with name as one string
for d in data:
    if d[1] not in author_dict:
        author_dict[d[1]]=[d[2]]
    else:
        author_dict[d[1]].append(d[2])
print(len(author_dict.keys()))

2079


In [12]:
author_dict['https://github.com/google/tf_mesh_renderer']

['Kyle Genova',
 'Forrester Cole',
 'Aaron Maschinot',
 'Aaron Sarna',
 'Daniel Vlasic',
 'William T. Freeman']

In [13]:
filename_mini = 'scraped_data_mini.txt' # use full version later

In [14]:
paper_data = {}

with open(filename) as fp:
    line = fp.readline().split()
    current_repo = line[0]
    contributors = [line[1:]]
    while line:
        line = fp.readline().split()
        if len(line) and line[0]==current_repo:
            contributors.append(line[1:])
        else:
            print(f'Doing {current_repo} with {len(contributors)} contributors')
            print(f'Authors: {author_dict[current_repo]}')
            paper_data[current_repo] = {}
            for cont in contributors:
                author_name = cont[1].replace("_"," ")
                found_name = author_name in author_dict[current_repo]
                print(f"{author_name} in {found_name}")
                if found_name:
                    paper_data[current_repo][author_name] = cont[2:]
            # after saving previous authors, load new repo
            current_repo = line[0]
            contributors = [line[1:]]                    

Doing https://github.com/google/tf_mesh_renderer with 3 contributors
Authors: ['Kyle Genova', 'Forrester Cole', 'Aaron Maschinot', 'Aaron Sarna', 'Daniel Vlasic', 'William T. Freeman']
NULL in False
Kyle Genova in True
NULL in False
Doing https://github.com/twke18/Adaptive_Affinity_Fields with 3 contributors
Authors: ['Tsung-Wei Ke', 'Jyh-Jing Hwang', 'Ziwei Liu', 'Stella X. Yu']
Tsung-Wei Ke in True
Jyh-Jing Hwang in True
Tsung-Wei Ke in True
Doing https://github.com/VladyslavUsenko/basalt-mirror with 3 contributors
Authors: ['Vladyslav Usenko', 'Nikolaus Demmel', 'Daniel Cremers', 'David Schubert', 'Thore Goll', 'Nikolaus Demmel', 'Vladyslav Usenko', 'Jörg Stückler', 'Daniel Cremers', 'Vladyslav Usenko', 'Nikolaus Demmel', 'David Schubert', 'Jörg Stückler', 'Daniel Cremers', 'Christiane Sommer', 'Vladyslav Usenko', 'David Schubert', 'Nikolaus Demmel', 'Daniel Cremers']
Vladyslav Usenko in True
Nikolaus Demmel in True
NULL in False
Doing https://github.com/new-okaerinasai/bwgan_pytorc

KeyError: 'https://github.com/benedekrozemberczki/karateclub'

In [15]:
len(paper_data)

212

### Creating graph

In [16]:
from rdflib import Graph, ConjunctiveGraph, Literal, BNode, Namespace, RDF, URIRef
from rdflib.namespace import DC, FOAF

In [ ]:
g = Graph()


### Full scraping approach

In [61]:
paper_data = {}
driver = webdriver.Firefox()

for paper in final_data:
    repo = paper[1]
    print(repo, paper[2]) # github repo and authors
    driver.get(f'{repo}/graphs/contributors')
    time.sleep(6) # wait for contribution graph to load
    website = BeautifulSoup(driver.page_source, 'html')
    boxes = website.find_all('h3', {'class':'border-bottom p-2 lh-condensed'})
    for user in boxes[0:10]:   # take just top 10 contributors, some repos might have 100+ contributors
        username = user.find('a', {'class':'text-normal', 'data-hovercard-type':'user'}).text.strip()
        info_line = user.find_all(('span', {'class':'cmeta'}))
        
        scrape = rest_get(f'https://github.com/{username}')
        soup = BeautifulSoup(scrape.content, 'html.parser')
        name = soup.find('span', class_='p-name vcard-fullname d-block overflow-hidden').text.strip()
        bio = soup.find('div', class_='p-note user-profile-bio mb-2 js-user-profile-bio').text.strip()
        print(f'name: {name} and {type(name)}, bio: {bio}')
        
        if name!="" and name in paper[2]:
            if paper[0] not in paper_data:
                paper_data[paper[0]]= [paper[1], {}]
            #print(info_line[1])
            commits = info_line[1].find('a').text.strip().split()[0]
            added = info_line[1].find('span', {'class':"text-green text-normal"}).text.strip().split()[0]
            deleted = info_line[1].find('span', {'class':"text-red text-normal"}).text.strip().split()[0]
            paper_data[paper[0]][1][name] = [commits, added, deleted]
            print(f'User {username}: commits {commits} , changed lines + {added} - {deleted}')
    print()

https://github.com/nianticlabs/monodepth2 ['Oisin Mac Aodha', 'Michael Firman', 'Gabriel J. Brostow']
https://github.com/mdfirman
name:  and <class 'str'>, bio: 
https://github.com/mrharicot
name: Clément Godard and <class 'str'>, bio: 
https://github.com/daniyar-niantic
name: Daniyar Turmukhambetov and <class 'str'>, bio: Senior Research Scientist at Niantic

https://github.com/google/tf_mesh_renderer ['Kyle Genova', 'Forrester Cole', 'Aaron Maschinot', 'Aaron Sarna', 'Daniel Vlasic', 'William T. Freeman']
https://github.com/fcole
name:  and <class 'str'>, bio: 
https://github.com/kylegenova
name: Kyle Genova and <class 'str'>, bio: 
User kylegenova: commits 2 , changed lines + 18 - 1
https://github.com/TerisGH
name:  and <class 'str'>, bio: 

https://github.com/Panda-Peter/visda2019-multisource ['Yingwei Pan', 'Yehao Li', 'Qi Cai', 'Yang Chen', 'Ting Yao']
https://github.com/Panda-Peter
name:  and <class 'str'>, bio: Yingwei Pan

https://github.com/yanggeng1995/GAN-TTS ['Mikołaj Bińk


https://github.com/DylanWusee/pointconv_pytorch ['Charles R. Qi', 'Hao Su', 'Kaichun Mo', 'Leonidas J. Guibas']


WebDriverException: Message: Failed to decode response from marionette


In [47]:
paper_data

{'https://paperswithcode.com/paper/unsupervised-training-for-3d-morphable-model': ['https://github.com/google/tf_mesh_renderer',
  {'Kyle Genova': ['2', '18', '1']}],
 'https://paperswithcode.com/paper/adaptive-affinity-fields-for-semantic': ['https://github.com/twke18/Adaptive_Affinity_Fields',
  {'Tsung-Wei Ke': ['2', '24', '6'], 'Jyh-Jing Hwang': ['9', '31', '26']}],
 'https://paperswithcode.com/paper/improving-information-retrieval-results-for': ['https://github.com/adelra/query-expansion',
  {'Adel Rahimi': ['2', '99,652', '0']}],
 'https://paperswithcode.com/paper/the-double-sphere-camera-model': ['https://github.com/VladyslavUsenko/basalt-mirror',
  {'Vladyslav Usenko': ['227', '66,951', '10,162'],
   'Nikolaus Demmel': ['24', '662', '1,219']}],
 'https://paperswithcode.com/paper/the-tum-vi-benchmark-for-evaluating-visual': ['https://github.com/VladyslavUsenko/basalt-mirror',
  {'Vladyslav Usenko': ['227', '66,951', '10,162'],
   'Nikolaus Demmel': ['24', '662', '1,219']}],
 'ht

In [3]:
repos = ['https://github.com/poloclub/cnn-explainer', 'https://github.com/openai/jukebox']

In [4]:
driver = webdriver.Firefox()

for repo in repos:
    print(repo)
    driver.get(f'{repo}/graphs/contributors')
    time.sleep(2)
    #delay of some kind wait for load time.sleep(3) or selenium wait for an element to be visible
    website = BeautifulSoup(driver.page_source, 'html')
    boxes = website.find_all('h3', {'class':'border-bottom p-2 lh-condensed'})
    for user in boxes:
        username = user.find('a', {'class':'text-normal', 'data-hovercard-type':'user'}).text.strip()
        info_line = user.find_all(('span', {'class':'cmeta'}))
        #print(info_line[1])
        commits = info_line[1].find('a').text.strip().split()[0]
        added = info_line[1].find('span', {'class':"text-green text-normal"}).text.strip().split()[0]
        deleted = info_line[1].find('span', {'class':"text-red text-normal"}).text.strip().split()[0]
        print(f'User {username}: commits {commits} , changed lines + {added} - {deleted}')
        scrape = rest_get(f'https://github.com/{username}')
        soup = BeautifulSoup(scrape.content, 'html.parser')
        name = soup.find('span', class_='p-name vcard-fullname d-block overflow-hidden').text.strip()
        bio = soup.find('div', class_='p-note user-profile-bio mb-2 js-user-profile-bio').text.strip()
        print(f'name: {name}, bio: {bio}')
        print()

https://github.com/poloclub/cnn-explainer
User xiaohk: commits 189 , changed lines + 35,562 - 14,943
https://github.com/xiaohk
name: Jay Wang, bio: Machine Learning PhD student at Georgia Tech.


Life is not always 🦄 and 🌈, but my favorite emoji is 😉

User rturko3: commits 58 , changed lines + 2,870 - 950
https://github.com/rturko3
name: , bio: CS Undergraduate @ Georgia Tech

User oshaikh13: commits 8 , changed lines + 117 - 74
https://github.com/oshaikh13
name: Omar Shaikh, bio: no u

https://github.com/openai/jukebox
User prafullasd: commits 49 , changed lines + 54,045 - 301
https://github.com/prafullasd
name: Prafulla Dhariwal, bio: 

User heewooj: commits 24 , changed lines + 393 - 344
https://github.com/heewooj
name: Heewoo Jun, bio: 

User jongwook: commits 5 , changed lines + 18 - 13
https://github.com/jongwook
name: Jong Wook Kim, bio: 

User mcleavey: commits 4 , changed lines + 964 - 3
https://github.com/mcleavey
name: Christine Payne, bio: OpenAI Researcher. Stanford Master